## Dashboard for exploring the glider section along with the surface properties

Here we combine the glider plots from the glider_trajectory_plot.ipynb and glider_vertical_section_plot.ipynb. 

In [ ]:
import hvplot.xarray 
import numpy as np
import panel as pn
import xarray as xr
import holoviews as hv
import geoviews as gv
from holoviews import opts
from holoviews import streams
import matplotlib.pyplot as plt
import geopandas
import pandas as pd
import hvplot.pandas  # noqa
import param
import cartopy.crs as ccrs

In [ ]:
# Load in surface variables
data_dir = './data/'
ds_ssh  = xr.open_dataset(data_dir + 'SSH_sogos_processed.nc')
ds_fsle = xr.open_dataset(data_dir + 'FSLE_sogos_processed.nc')

ds_659_locs = xr.open_dataset(data_dir + '659_locs.nc')
ds_660_locs = xr.open_dataset(data_dir + '660_locs.nc')

ds_ssh  = ds_ssh.assign_coords(days = ds_ssh.days)
ds_fsle = ds_fsle.assign_coords(days = ds_fsle.days)

# convert to pandas dataframe as it is much easier to handle in holoviz for traj data.
ds_659_pd = ds_659_locs.to_dataframe()
ds_660_pd = ds_660_locs.to_dataframe()

In [ ]:
# Load in vertical section

ds_659_Tgrid = xr.open_dataset(data_dir + '659_Tgrid.nc')
ds_660_Tgrid = xr.open_dataset(data_dir + '660_Tgrid.nc')

ds_659_Tgrid_anom = xr.open_dataset(data_dir + '659_Tgrid_anomaly.nc')
ds_660_Tgrid_anom = xr.open_dataset(data_dir + '660_Tgrid_anomaly.nc')

ds_659_Dgrid = xr.open_dataset(data_dir + '659_Dgrid.nc')
ds_660_Dgrid = xr.open_dataset(data_dir + '660_Dgrid.nc')

ds_659_Dgrid_anom = xr.open_dataset(data_dir + '659_Dgrid_anomaly.nc')
ds_660_Dgrid_anom = xr.open_dataset(data_dir + '660_Dgrid_anomaly.nc')

In [ ]:
glider_nums = ['sg659', 'sg660']

surface_var_map = {
    'SSH' : ds_ssh['adt'],
    'SSHA': ds_ssh['sla'],
    'FSLE': ds_fsle['fsle_max']
    }

glider_vars = list(ds_659_Tgrid.keys()) # just need to do once bevause all data sets have same variables
cmap_options = plt.colormaps()

var_select_map = {
            'oxygen': {
                'bin_range': (175, 375), # remove these since we don't use these anymore 
                'cmap_sel': 'YlOrBr'
            },
            'Chl': {
                'bin_range': (0, 0.5),
                'cmap_sel': 'Greens'
            },
            'salinity': {
                'bin_range': (33.75, 35),
                'cmap_sel': 'YlGnBu'
            },
            'temperature': {
                'bin_range': (0,4),
                'cmap_sel': 'RdBu_r'
            },
            'potdens': {
                'bin_range': (1026.8, 1027.8),
                'cmap_sel': 'Purples'
            }
        }
# For future versions would be nice if some of these things are read in from the
# attributes. 

glider_map = {
            'sg659': {'Time grid': ds_659_Tgrid, 'Distance grid': ds_659_Dgrid, 'loc': ds_659_pd},
            'sg660': {'Time grid': ds_660_Tgrid, 'Distance grid': ds_660_Dgrid, 'loc': ds_660_pd},
        }

glider_map_anom = {
            'sg659': {'Time grid': ds_659_Tgrid_anom, 'Distance grid': ds_659_Dgrid_anom},
            'sg660': {'Time grid': ds_660_Tgrid_anom, 'Distance grid': ds_660_Dgrid_anom},
        }


In [ ]:
class GliderParams(param.Parameterized):
    # Container for all the parameters for the widgets, 
    # and some default methods
    
    surface_var       = param.Selector(surface_var_map.keys(), default='SSH',
                                label='Surface Field', precedence=0)
    glider_num        = param.Selector(glider_map.keys(), default='sg659',
                                label='Glider Num', precedence=0)
    time_slider       = param.Range(label='Days in 2019', 
                             bounds=(119, 205), 
                             default=(119, 135), precedence=3)
    alpha_slider      = param.Magnitude(label='Transparency', precedence=4)
    glider_grid       = param.Selector(['Time grid', 'Distance grid'], default='Time grid', 
                                label='Grid Type', precedence=0)
    glider_var        = param.Selector(glider_vars, default='temperature', 
                                label='Glider Variable', precedence=1)
    var_colormap      = param.Selector(default='RdBu_r', objects=cmap_options, 
                                  label='Glider Section Colormap', precedence=2)
    distance_slider   = param.Range(label='Along Track Distance',
                                 bounds=(0, 2e3), default=(0, 400), 
                                 precedence=-1) # start with a negative precedence, in accordance with default being Tgrid
    anomaly_boolean   = param.Boolean(default=False, label='Anomaly', precedence=3)
    density_boolean   = param.Boolean(default=True, label='Show Density Contours', precedence=4)
    density_range     = param.Range(label='Density range', bounds=(1026.8, 1027.9), default=(1026.8, 1027.9),precedence=10)
    density_gradation = param.Integer(label='Density levels', default=11, bounds=(2, 21),precedence=10)
    
    # function to not have active toolbars by default
    def _set_tools(self, plot, element):
        plot.state.toolbar.active_drag = None
        plot.state.toolbar.active_inspect = None
    
    # function to update default colormap choices with changing variables
    @param.depends('glider_var', watch=True)
    def _update_colormap(self):
        self.var_colormap = var_select_map[self.glider_var]['cmap_sel']
    
    # The next couple of functions toggle the widgets visible or not. 
    @param.depends('density_boolean', watch=True)
    def _update_density_widgets(self):
        # helps to hide widgets when they are not being used. 
        if self.density_boolean:
            self.param.density_range.precedence=10
            self.param.density_gradation.precedence=10
        else:
            self.param.density_range.precedence=-1
            self.param.density_gradation.precedence=-1
            
    @param.depends('glider_grid', watch=True)
    def _update_grid_widgets(self):
        if self.glider_grid == 'Time grid':
            self.param.time_slider.precedence=3
            self.param.distance_slider.precedence=-1
        else: 
            self.param.time_slider.precedence=-1
            self.param.distance_slider.precedence=3

In [ ]:
class GliderTrajectoryPlot(GliderParams):
    # Function to plot trajectories
    @param.depends('glider_num', 'time_slider', 'distance_slider')
    def plot_traj(self):
        time_rng = self.time_slider
        dist_rng = self.distance_slider
        
        ###
        # For the selected glider do the proper time vs distance conversion
        # but for the unselected glider and surface plots always stick to the
        # corresponding time 
        ds = glider_map[self.glider_num]['loc']
        if self.glider_grid=='Time grid':
            ds_tsel = ds.loc[(ds.days>=time_rng[0]) & (ds.days<=time_rng[1])]
            dsel = (ds_tsel.iloc[0].distance, ds_tsel.iloc[-1].distance)
            self.distance_slider = dsel
        else:
            ds_tsel = ds.loc[(ds.distance>=dist_rng[0]) & (ds.distance<=dist_rng[1])]
            if int(ds_tsel.iloc[-1].days)<=205: # since the netcdf files for surface fields don't have a 206
                tsel = (int(ds_tsel.iloc[0].days), int(ds_tsel.iloc[-1].days))
            else:
                tsel = (int(ds_tsel.iloc[0].days), int(205))
            self.time_slider = tsel
        ###
        
        time_rng = self.time_slider # make sure time_rng has the most up to date values
        
        traj = {}
        for glid in glider_nums:
            ds = glider_map[glid]['loc']
            ds_tsel = ds.loc[(ds.days>time_rng[0]) & (ds.days<time_rng[1])]
        
            traj[glid] = ds_tsel.hvplot.points(geo=True,  x='longitude', y='latitude', 
                                               hover=True, hover_cols=['days'], 
                                               size=1)
            # setting PlateCarree as projection makes the hover cols show up properly, but then the bathy disappears.

        traj[self.glider_num].opts(size=2.5)
        
        return traj['sg659']*traj['sg660']
    
    # Function to plot tiles
    def surf_tiles(self):
        gebco_tiles = 'https://tiles.arcgis.com/tiles/C8EMgrsFcRFL6LrL/arcgis/rest/services/GEBCO_basemap_NCEI/MapServer/tile/{Z}/{Y}/{X}'
        return gv.WMTS( gebco_tiles )
    
    # function to plot vector field
    @param.depends('time_slider')
    def surf_vec(self):
        time_sel = self.time_slider[1] # show map for last day on time slider
        
        return ds_ssh.where(ds_ssh.days==time_sel, drop=True).squeeze('time'
                    ).hvplot.vectorfield(x='longitude', y='latitude', angle='angle', mag='mag',
                                        geo=True, hover=False).opts(magnitude='mag')
    
    #function to plot surface field 
    @param.depends('surface_var', 'time_slider', 'alpha_slider')
    def plot_surface(self):
        time_sel = self.time_slider[1] # show map for last day on time slider
        
        ds_all = surface_var_map[self.surface_var]
        ds = ds_all.where(ds_all.days==time_sel, drop=True).squeeze('time')
        if self.surface_var == 'FSLE':    
            surf_plot = ds.hvplot.image(geo=True)
            surf_plot.opts(clim=(-0.6,0), cmap='Blues_r', clabel='FSLE')
        elif self.surface_var == 'SSH':
            surf_plot = ds.hvplot.image(geo=True)
            surf_plot.opts(clim=(-1,0), cmap='cividis', clabel='SSH')
        else: 
            surf_plot = ds.hvplot.image(geo=True)
            surf_plot.opts(clim=(-0.3,0.3), cmap='RdBu_r', clabel='SSHA')
        
        surf_plot.opts(frame_width=450, alpha=self.alpha_slider, tools=['hover'], hooks=[self._set_tools])

        return surf_plot
        
    def view(self):
        return hv.DynamicMap(self.plot_surface)*hv.DynamicMap(self.surf_tiles
                                        )*hv.DynamicMap(self.surf_vec)*hv.DynamicMap(self.plot_traj)

In [ ]:
class GliderVerticalSectionPlot(GliderParams):
    
    # function to make density contours
    @param.depends('density_range', 'density_gradation', 'glider_grid','glider_num')
    def density_contours(self):
        #print('in contour')
        contour = glider_map[self.glider_num][self.glider_grid]['potdens'].hvplot.contour(flip_yaxis=True, 
                                                                     levels=np.linspace(self.density_range[0],
                                                                                        self.density_range[1],
                                                                                        self.density_gradation)
                                                                       ).opts(tools=[])
        return contour
    
    # function to make the image for the glider section
    @param.depends('anomaly_boolean', 'glider_grid', 'glider_num', 'glider_var')
    def glider_image(self):
        # Change the data set if wanting to plot anomaly
        if self.anomaly_boolean:
            glid_ds = glider_map_anom
        else:
            glid_ds = glider_map

        # plot the image in Distance or Time
        if self.glider_grid=='Distance grid':
            image = hv.Image( (glid_ds[self.glider_num][self.glider_grid].distance, 
                           glid_ds[self.glider_num][self.glider_grid].pressure,
                           glid_ds[self.glider_num][self.glider_grid][self.glider_var]), 
                             ['Distance [km]', 'Pressure [dBar]'], self.glider_var)

        else:
            image = hv.Image( (glid_ds[self.glider_num][self.glider_grid].time, 
                           glid_ds[self.glider_num][self.glider_grid].pressure,
                           glid_ds[self.glider_num][self.glider_grid][self.glider_var]), 
                             ['Time [days]', 'Pressure [dBar]'], self.glider_var)
        
        # estimate the color range so that outliers don't create problems
        bin_range = np.nanpercentile(glid_ds[self.glider_num][self.glider_grid][self.glider_var], [.5,99.5])
        
        # set properties for image like colorbar etcs.
        image = image.opts(opts.Image(
                        colorbar=True,
                        cmap=self.var_colormap,
                        invert_yaxis=True,
                        clim=(bin_range[0], bin_range[1]),
                        width=800,
                        tools=['hover'], hooks=[self._set_tools]
                        ) )
        return image
    
    #@param.depends('distance_slider', 'time_slider', 'glider_grid', 'density_boolean')
    def viewable(self):
        image = hv.DynamicMap(self.glider_image).hist()
                
        title = str(test.time_slider[0]) +'-'+ str(test.time_slider[1])+ ' Days & ' + str(int(test.distance_slider[0]))+'-'+str(int(test.distance_slider[1])) + ' km'
        
        if self.glider_grid=='Distance grid':
            image.opts(opts.Image(xlim = self.distance_slider, title=title))
        else:
            image.opts(opts.Image(xlim = self.time_slider, title=title))

        # add the density contours or not. 
        if self.density_boolean:
            return image*hv.DynamicMap(self.density_contours)
        else:
            return image
            

In [ ]:
class GliderCombinedPlot(GliderTrajectoryPlot, GliderVerticalSectionPlot):
    pass

In [ ]:
test = GliderCombinedPlot()

In [ ]:
test_panel = pn.Column(pn.Row(pn.Param(test.param, name=''),
                              pn.Column(pn.panel('## Southern Ocean Glider Observations of the Submesoscales\n Interactive dashboard to explore glider data collected in the Southern Ocean (zoom out on top panel to see exact location) during May-August 2019'),
                                        test.view())
                             ), 
                        test.viewable) #note that view() is passed vs viewable. 

In [ ]:
test_panel

In [ ]:
# Work locally 
# test_panel.show()

# Work on binder
test_panel.servable()